In [ ]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np

In [ ]:
URL = 'https://apteka.ru/norilsk/sym/leka/'
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0', 'accept': '*/*' }

def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r

def get_content(html):
    tablet = pd.DataFrame()
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='catalog-card card-flex')
    
    def try_price(item):     
        try:
            k = item.find_all('span', class_ = 'moneyprice__roubles')[0].get_text(strip=True)
        except:
            k = None
        return k

    def try_maker(item):
            try:
                a = item.find('span', class_ = 'emphasis').get_text(strip=True)
            except:
                a = None
            return a
    
    for item in items: 
        tablet = tablet.append(
                    {   'name' : item.find('a', class_ = 'catalog-card__name emphasis').get_text(strip=True),
                        'price' : try_price(item),
                        'maker' : try_maker(item),
                        'link' : 'https://apteka.ru' + item.find('a', class_='catalog-card__name emphasis').get('href'),
                        'form_med' : ''.join([x.get_text(strip=True) for x in item.find_all('div', class_ = 'card-param') if x.find('svg', class_='icon icon--properties_other') is not None ]),
                        'dose' : ''.join([x.get_text(strip=True) for x in item.find_all('div', class_ = 'card-param') if x.find('svg', class_='icon icon--properties_dose') is not None ]),
                        'quant_in_pac' : ''.join([x.get_text(strip=True) for x in item.find_all('div', class_ = 'card-param') if x.find('svg', class_='icon icon--properties_count') is not None ])
                    }, ignore_index=True
                )



    return tablet

def get_pages_count(html):
    soup = BeautifulSoup(html, 'html.parser')
    pagination = soup.find_all('a', class_='pager-v3-item')
    return len(pagination)

def parse():
    types = ['gorm/?withprice=1', 'derm/?withprice=1', 'diha/?withprice=1', 'kost/?withprice=1', 'krov/?withprice=1', 'moche/?withprice=1',
    'nerv/?withprice=1' ,'pishe/?withprice=1','prep/?withprice=1','prot/?withprice=1','proti/?withprice=1','protiv/?withprice=1','proch/?withprice=1','serd/?withprice=1']
    tb = pd.DataFrame()
    
    for type in types:
        
        html = get_html(URL + type)
        count_page = get_pages_count(html.text)

        if html.status_code == 200:               
            for page in range(1, count_page  + 1):
                html = get_html(URL + type, params= f'&page={page}')
                tab = get_content(html.text)
                tb = tb.append(tab).reset_index(drop=True)
                
        else:
            print('Error')
    return tb


tab = parse()



In [ ]:
tab


,name,price,maker,link,form_med,dose,quant_in_pac
0,ДИПРОСПАН,547,Шеринг-Плау Лабо Н.В.,https://apteka.ru/norilsk/product/diprospan-00...,СУСП Д/ИН,Доз. 2 мг + 5 мг/МЛ,1 шт в уп.
1,L-ТИРОКСИН 50 БЕРЛИН-ХЕМИ,108,Берлин-Хеми АГ,https://apteka.ru/norilsk/product/l-tiroksin-5...,таблетки,,50 шт в уп.
2,ДЕСМОПРЕССИН,1 812,ОЗОН ООО,https://apteka.ru/norilsk/product/desmopressin...,таблетки,"Доз. 0,2МГ",30 шт в уп.
3,ДИПРОМЕТА,574,К.О.Ромфарм Компани С.Р.Л.,https://apteka.ru/norilsk/product/diprometa-00...,СУСП Д/ИН,"Доз. 0,007/МЛ",1 шт в уп.
4,ПРЕДНИЗОЛОН,122,"Гедеон Рихтер Румыния А.О./ ""Гедеон Рихтер-РУС...",https://apteka.ru/norilsk/product/prednizolon-...,таблетки,,100 шт в уп.
5,ДЕКСАМЕТАЗОН,135,Эллара ООО,https://apteka.ru/norilsk/product/deksametazon...,,,
6,КАЛИЯ ЙОДИД,102,Алиум АО,https://apteka.ru/norilsk/product/kaliya-jodid...,,,
7,ПОЛЬКОРТОЛОН,563,"Адамед Фарма, АО",https://apteka.ru/norilsk/product/polkortolon-...,таблетки,,50 шт в уп.
8,"НАБОР 1 уп. ПРЕДНИЗОЛОН 0,5% 15,0 МАЗЬ Д/НАР П...",355,None,https://apteka.ru/norilsk/product/61a7706bb92b...,,,
9,L-ТИРОКСИН 100 БЕРЛИН-ХЕМИ,125,Берлин-Хеми АГ,https://apteka.ru/norilsk/product/l-tiroksin-1...,,,
